In [1]:
import numpy as np
import pandas as pd
import refinitiv.dataplatform.eikon as ek

In [2]:
import refinitiv.data as rd
from refinitiv.data.errors import RDError
from datetime import timedelta
from datetime import datetime
rd.open_session()

<refinitiv.data.session.Definition object at 0x13dfd51f0 {name='workspace'}>

## Предобработка данных

* ISIN для базового актива в виде строки
* Дата истечения для опционов в виде строки
* Цены страйков, для которых нужно получить котировки опционов, в виде списка целых чисел
* Типы опционов (колл/пут), для которых нужно получить котировки, в виде списка строк (допустимые значения — 'C' и 'P')

In [4]:
def get_exp_month(maturity, opt_type, strike = None, opra=False):
   
   maturity = pd.to_datetime(maturity)
   # define option expiration identifiers
   ident = {'1': {'exp': 'A','C': 'A', 'P': 'M'}, 
          '2': {'exp': 'B', 'C': 'B', 'P': 'N'}, 
          '3': {'exp': 'C', 'C': 'C', 'P': 'O'}, 
          '4': {'exp': 'D', 'C': 'D', 'P': 'P'},
          '5': {'exp': 'E', 'C': 'E', 'P': 'Q'},
          '6': {'exp': 'F', 'C': 'F', 'P': 'R'},
          '7': {'exp': 'G', 'C': 'G', 'P': 'S'}, 
          '8': {'exp': 'H', 'C': 'H', 'P': 'T'}, 
          '9': {'exp': 'I', 'C': 'I', 'P': 'U'}, 
          '10': {'exp': 'J', 'C': 'J', 'P': 'V'},
          '11': {'exp': 'K', 'C': 'K', 'P': 'W'}, 
          '12': {'exp': 'L', 'C': 'L', 'P': 'X'}}
   
   # get expiration month code for a month
   if opt_type.upper() == 'C':
       exp_month = ident[str(maturity.month)]['C']
       
   elif opt_type.upper() == 'P':
       exp_month = ident[str(maturity.month)]['P']
   
   if opra and strike > 999.999:
       exp_month = exp_month.lower()
           
   return ident, exp_month

In [5]:
# check if option is expired
def check_expiry(ric, maturity, ident):
   maturity = pd.to_datetime(maturity)
   if maturity < datetime.now():
       ric = ric + '^' + ident[str(maturity.month)]['exp'] + str(maturity.year)[-2:]
   return ric

In [6]:
def request_prices(ric):
   prices = []
   try:    
       prices = rd.get_history(ric, fields = ['BID','ASK','TRDPRC_1','SETTLE'])
   except RDError as err:
       print(f'Constructed ric {ric} -  {err}')
   return prices

In [7]:
def get_ric_opra(asset, maturity, strike, opt_type):
   maturity = pd.to_datetime(maturity)
   
   # trim underlying asset's RIC to get the required part for option RIC
   if asset[0] == '.': # check if the asset is an index or an equity
       asset_name = asset[1:] # get the asset name - we remove "." symbol for index options
   else:
       asset_name = asset.split('.')[0] # we need only the first part of the RICs for equities
       
   ident, exp_month = get_exp_month(maturity, opt_type, strike = strike, opra=True)
   # get strike prrice
   if type(strike) == float:
       int_part = int(strike)
       dec_part = str(str(strike).split('.')[1])
   else:
       int_part = int(strike)
       dec_part = '00'
   if len(dec_part) == 1:
       dec_part = dec_part + '0'
   if int(strike) < 10:
       strike_ric = '00' + str(int_part) + dec_part
   elif int_part >= 10 and int_part < 100:
       strike_ric = '0' + str(int_part) + dec_part
   elif int_part >= 100 and int_part < 1000:
       strike_ric = str(int_part) + dec_part
   elif int_part >= 1000 and int_part < 10000:
       strike_ric = str(int_part) + '0'
   elif int_part >= 10000 and int_part < 20000:
       strike_ric = 'A' + str(int_part)[-4:]
   elif int_part >= 20000 and int_part < 30000:
       strike_ric = 'B' + str(int_part)[-4:]      
   elif int_part >= 30000 and int_part < 40000:
       strike_ric = 'C' + str(int_part)[-4:]
   elif int_part >= 40000 and int_part < 50000:
       strike_ric = 'D' + str(int_part)[-4:]
       
   # build ric
   ric = asset_name + exp_month + str(maturity.day) + str(maturity.year)[-2:] + strike_ric + '.U'
   ric = check_expiry(ric, maturity, ident)
   
   prices = request_prices(ric)
   
   # return valid ric(s)
   if len(prices) == 0:
       print('RIC with specified parameters is not found')
   return ric, prices

In [54]:
ric, prices = get_ric_opra("AAPL.O", maturity, 215, 'C')

/Users/amir/anaconda3/envs/myenv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [37]:
prices

AAPLK292421500.U,BID,ASK,TRDPRC_1,SETTLE
Date,,,,
2024-10-10,16.65,20.0,18.05,<NA>
2024-10-11,16.35,17.3,17.89,<NA>
2024-10-14,18.7,20.65,19.5,<NA>
2024-10-15,21.85,22.45,22.53,<NA>
2024-10-16,19.5,20.25,18.75,<NA>
2024-10-17,19.8,20.55,19.88,<NA>
2024-10-18,22.45,22.7,23.24,<NA>
2024-10-21,23.65,24.15,23.8,<NA>
2024-10-22,22.85,23.35,23.19,<NA>


In [8]:
ric = 'AAPL.O'
maturity = '2024-11-29'
strikes = [215, 220, 225, 230, 235]
kinds = ['C', 'P']

In [18]:
price_dfs = []
for kind in kinds:
    assert kind in ['C', 'P']
    for strike in strikes:
        kind_str = 'call' if kind == 'C' else 'P'
        print(f'\nFetching prices for {kind_str} option with strike price {strike}')
        rics, prices_list = get_ric_opra(ric, maturity, strike, kind)
        print(ric, maturity, strike, kind)
        try:
            price_df = prices_list
            price_df = price_df.drop(['BID', 'ASK', 'SETTLE'], axis=1)
            price_df.columns = [f'{kind}{strike}']
            price_dfs.append(price_df)
        except IndexError:
            print('Skipping empty combination')

price_df = pd.concat(price_dfs, axis=1)


Fetching prices for call option with strike price 215


/Users/amir/anaconda3/envs/myenv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


AAPL.O 2024-11-29 215 C

Fetching prices for call option with strike price 220


/Users/amir/anaconda3/envs/myenv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


AAPL.O 2024-11-29 220 C

Fetching prices for call option with strike price 225


...

/Users/amir/anaconda3/envs/myenv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


AAPL.O 2024-11-29 225 P

Fetching prices for P option with strike price 230


/Users/amir/anaconda3/envs/myenv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


AAPL.O 2024-11-29 230 P

Fetching prices for P option with strike price 235
AAPL.O 2024-11-29 235 P


/Users/amir/anaconda3/envs/myenv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [47]:
price_df.tail(10)

,C215,C220,C225,C230,C235,P215,P220,P225,P230,P235
Date,,,,,,,,,,
2024-10-23,18.05,13.9,11.45,8.25,5.7,2.3,3.4,4.78,6.78,10.0
2024-10-24,18.75,14.57,11.15,8.2,5.7,2.3,3.35,4.76,6.8,9.2
2024-10-25,19.35,15.15,11.8,8.72,6.0,2.1,3.23,4.48,6.5,9.0
2024-10-28,21.35,16.9,13.15,9.6,6.8,1.69,2.61,3.6,5.34,7.6
2024-10-29,20.8,17.15,13.35,9.6,6.75,1.47,2.4,3.59,5.15,7.2
2024-10-30,18.25,14.42,10.5,7.6,5.24,2.26,3.4,5.0,7.0,9.5
2024-10-31,15.27,11.84,8.6,5.8,4.04,3.4,4.98,7.05,9.12,12.12
2024-11-01,11.06,8.0,5.15,3.05,1.64,2.86,4.4,6.85,9.54,13.64
2024-11-04,10.1,6.95,4.33,2.3,1.22,2.92,4.55,6.9,9.79,13.5


In [19]:
sdate = str(price_df.index.min().date())
edate = str(price_df.index.max().date())

## Базовый актив

In [20]:
stock = ek.get_timeseries(
    rics=["AAPL.O"],
    fields=['CLOSE'],
    start_date=sdate,
    end_date=edate,
    interval='daily'
)

stock = stock.reset_index()
stock.columns.name = None
stock.index = stock['Date']
stock.index.name = None
stock = stock.drop('Date', axis=1)
stock.columns = ['Underlying']

price_df = stock.join(price_df)

In [21]:
price_df

,Underlying,C215,C220,C225,C230,C235,P215,P220,P225,P230,P235
2024-10-10,229.04,18.05,14.41,11.23,8.65,5.8,3.3,4.67,6.4,8.35,10.25
2024-10-11,227.55,17.89,14.1,10.95,7.55,5.46,3.06,4.35,5.95,8.5,10.8
2024-10-14,231.3,19.5,16.0,12.5,9.27,6.7,2.46,3.42,4.84,6.6,9.0
2024-10-15,233.85,22.53,19.5,14.05,10.62,7.83,2.19,3.07,4.2,5.9,8.07
2024-10-16,231.78,18.75,15.18,12.04,9.21,6.47,2.16,3.23,4.53,6.7,8.74
2024-10-17,232.15,19.88,16.0,12.28,9.22,6.51,2.09,2.9,4.23,5.9,8.45
2024-10-18,235.0,23.24,18.35,14.79,11.07,7.85,1.59,2.35,3.33,4.85,6.75
2024-10-21,236.48,23.8,18.94,14.87,11.28,8.5,1.39,1.99,2.92,4.25,6.08
2024-10-22,235.86,23.19,19.2,14.6,10.92,8.25,1.35,2.03,3.05,4.29,6.14
2024-10-23,230.76,18.05,13.9,11.45,8.25,5.7,2.3,3.4,4.78,6.78,10.0


In [13]:
price_df.head()

,Underlying,C215,C220,C225,C230,C235,P215,P220,P225,P230,P235
2024-10-10,229.04,18.05,14.41,11.23,8.65,5.8,3.3,4.67,6.4,8.35,10.25
2024-10-11,227.55,17.89,14.1,10.95,7.55,5.46,3.06,4.35,5.95,8.5,10.8
2024-10-14,231.3,19.5,16.0,12.5,9.27,6.7,2.46,3.42,4.84,6.6,9.0
2024-10-15,233.85,22.53,19.5,14.05,10.62,7.83,2.19,3.07,4.2,5.9,8.07
2024-10-16,231.78,18.75,15.18,12.04,9.21,6.47,2.16,3.23,4.53,6.7,8.74


In [17]:
price_df

,Date,Underlying,C215,C220,C225,C230,C235,P215,P220,P225,P230,P235
Date,,,,,,,,,,,,
0,2024-10-10,229.04,18.05,14.41,11.23,8.65,5.8,3.3,4.67,6.4,8.35,10.25
1,2024-10-11,227.55,17.89,14.1,10.95,7.55,5.46,3.06,4.35,5.95,8.5,10.8
2,2024-10-14,231.3,19.5,16.0,12.5,9.27,6.7,2.46,3.42,4.84,6.6,9.0
3,2024-10-15,233.85,22.53,19.5,14.05,10.62,7.83,2.19,3.07,4.2,5.9,8.07
4,2024-10-16,231.78,18.75,15.18,12.04,9.21,6.47,2.16,3.23,4.53,6.7,8.74
5,2024-10-17,232.15,19.88,16.0,12.28,9.22,6.51,2.09,2.9,4.23,5.9,8.45
6,2024-10-18,235.0,23.24,18.35,14.79,11.07,7.85,1.59,2.35,3.33,4.85,6.75
7,2024-10-21,236.48,23.8,18.94,14.87,11.28,8.5,1.39,1.99,2.92,4.25,6.08
8,2024-10-22,235.86,23.19,19.2,14.6,10.92,8.25,1.35,2.03,3.05,4.29,6.14


In [22]:
price_df.index.name = 'Date'
price_df = price_df.reset_index()
price_df.to_csv('..data/aapl.csv')